In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
import math
import time
import pyproj
import random
from shapely.geometry import LineString, CAP_STYLE
from PIL import Image
import PIL.Image
import matplotlib.pyplot as plt
import mercantile
from geotile import GeoTile
from gdal2tiles import generate_tiles
#from gdal2tiles import gdalinfo
from cores import *
import os
from path import * 
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import platform
print(platform.system(),)
print(platform.machine(),)
print(platform.platform(),)
print(platform.uname(),)
print(platform.version(),)
print(platform.mac_ver(),)
#import rasterio
#from rasterio import features

Windows
AMD64
Windows-10-10.0.19045-SP0
uname_result(system='Windows', node='C118', release='10', version='10.0.19045', machine='AMD64')
10.0.19045
('', ('', '', ''), '')


In [2]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)

df = df.loc[(df.cor != 'humidity_rastro') & (df.cor != 'produzindo_rastro')]
df = df.drop(['Unnamed: 0'], axis = 1)
df.cor = df.cor.str.capitalize()

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs=WGS84)

application_color_list = ['App_L3', 'App_L2', 'App_L1', 'App_0', 'App_H1', 'App_H2', 'App_H3']
camada_objeto = 'App_rastro'
if len(gdf[gdf['cor'] == camada_objeto]['value'].dropna().unique()) > 1:
    print("Tiles - processar polígonos de velocidade")
    inicial = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(0.1)
    final = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(0.9)
    topo = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(1)
    n_intervalos = 7
    passo = (final - inicial) / (n_intervalos - 2)
    atual = inicial
    intervalos = []
    intervalos.append([0, inicial])
    for i in list(range(2, n_intervalos)):
        intervalos.append([atual, atual + passo])
        atual = atual + passo
    intervalos.append([final, topo + 1])
    pos_int = 0
    for intervalo in intervalos:
        gdf.loc[(gdf['cor'] == camada_objeto) & (gdf['value'] >= intervalo[0]) & (gdf['value'] < intervalo[1]), 'cor'] = application_color_list[pos_int]
        pos_int = pos_int + 1
gdf = gdf.reset_index(drop=True)


gdf = gdf.to_crs(MERCATOR)
mask =[]
for i in gdf.geometry: 
    mask.append(type(i)==type(gdf.geometry.loc[0]))
gdf.geometry.loc[mask] = gdf.geometry.loc[mask].buffer(1,  cap_style=CAP_STYLE.square)

source = ogr.Open(gdf.to_crs(WGS84).to_json())

#gdf


Tiles - processar polígonos de velocidade


In [3]:
source_ds = ogr.GetDriverByName("Memory").CopyDataSource(source, "")
source_layer = source_ds.GetLayer(0)
source_srs = source_layer.GetSpatialRef()
x_min, x_max, y_min, y_max = source_layer.GetExtent()
numFeatures = source_layer.GetFeatureCount()



source_layer.CreateField(ogr.FieldDefn('Red', ogr.OFTReal))
source_layer.CreateField(ogr.FieldDefn('Green', ogr.OFTReal))
source_layer.CreateField(ogr.FieldDefn('Blue', ogr.OFTReal))


source_layer_def = source_layer.GetLayerDefn()
fd_id = source_layer_def.GetFieldIndex('cor')
r_id = source_layer_def.GetFieldIndex('Red')
g_id = source_layer_def.GetFieldIndex('Green')
b_id = source_layer_def.GetFieldIndex('Blue')


ldefn = source_layer.GetLayerDefn()
ldefn.GetFieldCount()
lsAtribNames = [field.name for field in source_layer.schema]

In [4]:
for feature in source_layer:
    feature.SetField(r_id,globals()[feature.GetField(fd_id)][0])
    source_layer.SetFeature(feature)
for feature in source_layer:
    feature.SetField(g_id,globals()[feature.GetField(fd_id)][1])
    source_layer.SetFeature(feature)
for feature in source_layer:
    feature.SetField(b_id,globals()[feature.GetField(fd_id)][2])
    source_layer.SetFeature(feature)

In [5]:
pix_size = 0.0000002
x_res = int((x_max - x_min) / pix_size)
y_res = int((y_max - y_min) / pix_size)
x_res, y_res

(3317, 15363)

In [7]:
target_ds = err = source_layer = None

In [6]:
#target_ds = gdal.GetDriverByName('MEM').Create('', x_res, y_res, 4, gdal.GDT_Byte)
target_ds = gdal.GetDriverByName('GTiff').Create('rasterize.tif', x_res, y_res, 4, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pix_size, 0, y_max, 0, -pix_size))

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'

if source_srs:
    target_ds.SetProjection(source_srs.ExportToWkt())
else:
    target_ds.SetProjection(proj)

#err = gdal.RasterizeLayer(target_ds, [1,2,3], source_layer, burn_values=(0xff, 0xff, 0xff))

err = gdal.RasterizeLayer(target_ds, [1], source_layer, options=["ATTRIBUTE=%s" % 'Red'])
err = gdal.RasterizeLayer(target_ds, [2], source_layer, options=["ATTRIBUTE=%s" % 'Green'])
err = gdal.RasterizeLayer(target_ds, [3], source_layer, options=["ATTRIBUTE=%s" % 'Blue'])
err = gdal.RasterizeLayer(target_ds, [4], source_layer, burn_values=[0xff])

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

target_ds = err = source_layer = None


RuntimeError: Unable to open rasterize.tif to obtain file list.

In [ ]:
img = Image.fromarray(target_ds.ReadAsArray().transpose(), mode="RGBA").rotate(90, expand=True).transpose(PIL.Image.FLIP_TOP_BOTTOM)

In [ ]:
options = {'zoom': (7, 19), 'resume': True, 'tmscompatible': True}
generate_tiles('rasterize.tif', 'output_dir', **options)

RuntimeError: `rasterize.tif' not recognized as a supported file format.